In [18]:
%matplotlib inline

import numpy as np
import scipy.io as sio
import matplotlib.pyplot as plt
import scipy.optimize as op
import math as math
from sklearn import svm, datasets

# use seaborn plotting defaults
import seaborn as sns; sns.set()

In [19]:
# Visulize data example
f = open('emailSample1.txt', 'r')
email = f.read()
print(email) 

> Anyone knows how much it costs to host a web portal ?
>
Well, it depends on how many visitors you're expecting.
This can be anywhere from less than 10 bucks a month to a couple of $100. 
You should checkout http://www.rackspace.com/ or perhaps Amazon EC2 
if youre running something big..

To unsubscribe yourself from this mailing list, send an email to:
groupname-unsubscribe@egroups.com




In [20]:
#data=f.read().replace('\n', '')   Might be usefull
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")
def processEmail(email):
    email.lower()
    email = strip_tags(email)    
    email = normalizeUrls(email)
    email = normalizeEmailAddr(email)
    email = normalizeDollar(email)
    email = normalizeNumbers(email)
    email = removeNonWords(email)
    email = removeWhiteSpace(email)
    
    email_split = email.split()
    email_stemmed = [stemmer.stem(word) for word in email_split ]
    
    return email_stemmed

from html.parser import HTMLParser

import re
def normalizeUrls(email):
    _link = re.compile(r'(?:(http://)|(www\.))(\S+\b/?)([!"#$%&\'()*+,\-./:;<=>?@[\\\]^_`{|}~]*)(\s|$)', re.I)
    output = re.sub(_link, 'httpaddr', email)
    return output

def normalizeEmailAddr(email):
    _link = re.compile("([a-z0-9!#$%&'*+\/=?^_`{|}~-]+(?:\.[a-z0-9!#$%&'*+\/=?^_`"
                    "{|}~-]+)*(@|\sat\s)(?:[a-z0-9](?:[a-z0-9-]*[a-z0-9])?(\.|"
                    "\sdot\s))+[a-z0-9](?:[a-z0-9-]*[a-z0-9])?)")
    output = re.sub(_link, 'httpaddr', email)
    return output

def normalizeDollar(email):
    _link = re.compile(r'([£$€])(\d+(?:\.\d{2})?)')
    #_link = re.compile(r'([£$€])')
    output = re.sub(_link, 'dollarnumb', email)
    return output

def normalizeNumbers(email):
    _link = re.compile("[-+]?\d+[\.]?\d*")
    output = re.sub(_link, 'number', email)
    return output

def removeNonWords(email):
    _link = re.compile('[^a-zA-Z\d\s]')
    output = re.sub(_link, '', email)
    return output

def removeWhiteSpace(email):
    test = " "   
    return test.join(email.split())
    
class MLStripper(HTMLParser):
    def __init__(self):
        super().__init__()
        self.reset()
        self.fed = []
    def handle_data(self, d):
        self.fed.append(d)
    def get_data(self):
        return ''.join(self.fed)

def strip_tags(html):
    s = MLStripper()
    s.feed(html)
    return s.get_data()

email_processed = processEmail(email)


f = open('vocab.txt', 'r')
vocab_list = f.read().split()

def vocabMapping(email_processed, vocab_list):
    email_map = np.zeros([vocab_list.shape[0]])
    matches = np.intersect1d(email_processed, vocab_list)
    for match in matches:
        itemindex = np.where(match==vocab_list)
        email_map[itemindex] = 1
        
    return email_map
        
email_map = vocabMapping(email_processed, words)




/usr/local/lib/python3.4/dist-packages/ipykernel/__main__.py:56: DeprecationWarning: The value of convert_charrefs will become True in 3.5. You are encouraged to set the value explicitly.


In [21]:
# ** This part does not use any of the above code ** #

# Load already processed email data, train an linear svm and print scores
emails = sio.loadmat('spamTrain.mat')
X = emails['X']
y = emails['y'].flatten()

model = svm.LinearSVC(C=0.1)
model.fit(X,y)
print("Training Data Score: ", model.score(X, y))

emails = sio.loadmat('spamTest.mat')
X = emails['Xtest']
y = emails['ytest'].flatten()
print("Test Data Score: ", model.score(X, y))

Training Data Score:  0.99975
Test Data Score:  0.992
